In [1]:
%load_ext autoreload
%autoreload 2

### **IMPORTING LIBRARY AND SETTING PATH**

In [2]:
# Needed Libraries
import numpy as np
from scipy import signal as sig
from Signal_Analysis_Class import heart_rate
import wfdb as wf
import pywt as WT
import matplotlib.pyplot as plt
import os
from multiprocessing.pool import Pool
from pydmd import HODMD
import warnings



# Specify the default path
os.chdir('..')
dir = os.getcwd()

# Global variables
results = []

### **FUNCTION DEFINITIONS**

In [3]:
def Signal_Filter(Signal):
    # Wavelet Tranformation
    Coeff = WT.wavedec(Signal, 'db3', level = 11)
    Coeff_Mod = []

    for i in range(0,11):
        if i > 3:
            Coeff_Mod.append(Coeff[i])
        else:
            Coeff_Mod.append(np.zeros(Coeff[i].shape))

    # Inverse Wavelet Transform
    Signal_WT = WT.waverec(Coeff_Mod, 'db3')

    # Notch Filter
    samp_freq = 1000
    notch_freq = 60.0
    quality_factor = 2

    b_notch, a_notch = sig.iirnotch(notch_freq, quality_factor, samp_freq)
    Signal_Notch =  sig.filtfilt(b_notch, a_notch, Signal_WT)

    # Fourth order Butterworth low pass filter at 100Hz
    sos = sig.butter(N = 4, Wn = 100,output ='sos', fs=1000)
    return sig.sosfilt(sos, Signal_Notch)

def Patient_Signals_Filter(Signals_Array):
    Signals_Filtered_Array = None

    for s in Signals_Array:
        Signal_Filtered = Signal_Filter(s)

        if str(type(Signals_Filtered_Array)) == "<class 'NoneType'>":
            Signals_Filtered_Array = Signal_Filtered
        else:
            Signals_Filtered_Array = np.vstack((Signals_Filtered_Array,Signal_Filtered))

    return Signals_Filtered_Array

def Beat_ind(Signal_Filterd_Array):

    Peak_ind = None

    for s in Signal_Filterd_Array:

        hr = heart_rate(s,1000)
        result = np.array(hr.find_r_peaks())

        result = np.unique(result[result > 0])
        result = result[1:len(result)-1]

        for i in range(1,len(result)-1):
            if result[i] > 300:
                valid_peaks = [result[i]]
                break

        for i in range(1,len(result)-1):
            if result[i] - valid_peaks[-1] > 250:
                valid_peaks.append(result[i])
        
        valid_peaks = np.array(valid_peaks)
        result = valid_peaks

        if str(type(Peak_ind)) == "<class 'NoneType'>":
                Peak_ind = np.array([result])
        else:
            if Peak_ind.shape[1] == result.shape[0]:
                Peak_ind = np.vstack((Peak_ind,result))
            elif Peak_ind.shape[1] > result.shape[0]:
                Peak_ind = np.vstack((Peak_ind[:,:result.shape[0]],result))
            elif Peak_ind.shape[1] < result.shape[0]:
                Peak_ind = np.vstack((Peak_ind,result[:Peak_ind.shape[1]]))
            else:
                print("Indexing error!!!")
                break
    
    return Peak_ind

def Beat_Signal_Extraction(Ori_signal, ind):

    Sample_Signal = None
    for n in range(12):
        if str(type(Sample_Signal)) == "<class 'NoneType'>":
            Sample_Signal = Ori_signal[n,int(ind[n]-250):int(ind[n]+400)]
        else:
            Sample_Signal = np.vstack((Sample_Signal,Ori_signal[n,int(ind[n]-250):int(ind[n]+400)]))
    return Sample_Signal

def Arg_X(Sample_signal):
    
    B = Sample_signal.flatten(order = 'F')
    N = (Sample_signal.shape[1] - 109) + 1

    Arg_Matrix = None

    for n in range(N):
        V = B[n*12:n*12 + 109*12]
        if str(type(Arg_Matrix)) == "<class 'NoneType'>":
            Arg_Matrix = V
        else:
            Arg_Matrix = np.vstack((Arg_Matrix,V))
    
    return Arg_Matrix.T

def FeatureExtraction_Beat(Signal_F, ind,n):

    A = Beat_Signal_Extraction(Signal_F, ind)
    B = Arg_X(A)

    X_DMD =  HODMD(svd_rank = 0, exact= True)
    X_DMD.fit(B)

    Lambda = X_DMD.eigs
    Modes = X_DMD.modes    

    M = int(Modes.shape[0]/12)

    Modes_restacked = np.zeros((12,Modes.shape[1]))

    for j in range(M):
        Modes_restacked = Modes_restacked + Modes[int(j*12):int(j*12 + 12),:]

    Modes_restacked = Modes_restacked/M

    Lambda_ind_u = np.where(np.abs(Lambda) > 1)
    Lambda_ind_s = np.where(np.abs(Lambda) < 1)

    Lambda_u = Lambda[Lambda_ind_u]
    Lambda_s = Lambda[Lambda_ind_s]

    Pho_u = Modes_restacked[:,Lambda_ind_u].reshape((12,Lambda_u.shape[0]))
    Pho_s = Modes_restacked[:,Lambda_ind_s].reshape((12,Lambda_s.shape[0]))

    C = Lambda_s.shape[0]
    D = Lambda_u.shape[0]

    R_N = D/(C+D)
    R_Lambda = np.sum(np.abs(Lambda_u))/(np.sum(np.abs(Lambda_u)) + np.sum(np.abs(Lambda_s)))

    Lam_min = np.min(np.abs(Lambda_s), initial = -1 )
    Lam_max = np.max(np.abs(Lambda_u), initial = -1 )

    R_M = np.sum(np.sum(np.abs(Pho_u), axis = 1))/(np.sum(np.sum(np.abs(Pho_u), axis = 1)) + np.sum(np.sum(np.abs(Pho_s), axis = 1)))
    R_P = np.sum(np.sum(np.angle(Pho_u), axis = 1))#/(np.sum(np.sum(np.angle(Pho_u), axis = 1)) +  np.sum(np.sum(np.angle(Pho_s), axis = 1)))
    

    M_s = np.sum(np.abs(Pho_s), axis=1)/C
    P_s = np.sum(np.angle(Pho_s), axis=1)/C

    M_u = np.sum(np.abs(Pho_u), axis=1)/D
    P_u = np.sum(np.angle(Pho_u), axis=1)/D

    print(f"n: {n}, Stable: {C}, Unstable: {D}")
    Feature_String = str(R_N) + "," + str(R_Lambda) + "," + str(Lam_min) + "," + str(Lam_max) + "," + str(R_M) + "," + str(R_P) + Numpy_arr_to_String(M_s) + Numpy_arr_to_String(P_s) + Numpy_arr_to_String(M_u) + Numpy_arr_to_String(P_u)
    return (str(n) + ',' + Feature_String)

def Numpy_arr_to_String(Arr):
    string = ''
    for no in Arr:
        string = string + ',' + str(no) 

    return string

def get_results(result):
    global results
    results.append(result)

def Patient_FeatureExtraction_Beat(patient, name):
    Signal_Array, fields = wf.rdsamp( dir + '/DATASET/' + patient + '/' + name)

    Signal_Array = Signal_Array.T
    Signal_Array = Signal_Array[:12,:]

    Signal_Filterd_Array = Patient_Signals_Filter(Signal_Array)

    Beat_index = Beat_ind(Signal_Filterd_Array)

    Peaks_count = Beat_index.shape[1]

    Result_File = []
    print(f"Patient: {patient} Audio ID: {name}")
    for n in range(Peaks_count):
        Result_File.append(patient + '_' + FeatureExtraction_Beat(Signal_Filterd_Array,Beat_index[:,n],n))
        
    return(Result_File)

### **LOADING DATASET**

In [4]:
# Specifying Dataset Directory
os.chdir(dir + '/DATASET')
Patients_List = os.listdir()
os.chdir(dir)

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]  
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    print(fields['comments'][4])


# Healthy control List
HC = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]  
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Healthy control' in fields['comments'][4]:
        HC.append([patient,F[0].replace('.dat','')])

# Myocardial infarction
MI = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]  
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Myocardial infarction' in fields['comments'][4]:
        MI.append([patient,F[0].replace('.dat','')])

# Cardiomyopathy/Heart failure
HF = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]  
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Cardiomyopathy' in fields['comments'][4] or 'Heart failure' in fields['comments'][4]:
        HF.append([patient,F[0].replace('.dat','')])

# Bundle branch block
BBB = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]  
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Bundle branch block' in fields['comments'][4]:
        BBB.append([patient,F[0].replace('.dat','')])

# Dysrhythmia
DA = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Dysrhythmia' in fields['comments'][4]:
        DA.append([patient,F[0].replace('.dat','')])

# Myocardial hypertrophy
MH = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Myocardial hypertrophy' in fields['comments'][4]:
        MH.append([patient,F[0].replace('.dat','')])

# Valvular heart disease
VH = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Valvular heart' in fields['comments'][4]:
        VH.append([patient,F[0].replace('.dat','')])

# Myocarditis
MC = []

for patient in Patients_List:
    os.chdir(dir + '/DATASET/' + patient) 
    Files = os.listdir() 
    F = [a for a in Files if a.endswith('.dat')]
    _, fields = wf.rdsamp( dir + '/DATASET/' + patient +'/' + F[0].replace('.dat',''))
    if 'Myocarditis' in fields['comments'][4]:
        MC.append([patient,F[0].replace('.dat','')])


Reason for admission: Heart failure (NYHA 2)
Reason for admission: Myocardial infarction
Reason for admission: Healthy control
Reason for admission: Healthy control
Reason for admission: Myocardial infarction
Reason for admission: Dysrhythmia
Reason for admission: Healthy control
Reason for admission: Healthy control
Reason for admission: Healthy control
Reason for admission: Myocardial infarction
Reason for admission: Myocardial infarction
Reason for admission: Myocardial infarction
Reason for admission: Healthy control
Reason for admission: Myocardial infarction
Reason for admission: Myocardial infarction
Reason for admission: Myocardial infarction
Reason for admission: Healthy control
Reason for admission: Myocardial infarction
Reason for admission: n/a
Reason for admission: Myocardial infarction
Reason for admission: Myocardial infarction
Reason for admission: Dysrhythmia
Reason for admission: Myocardial infarction
Reason for admission: n/a
Reason for admission: Myocardial infarcti

### **SEGEMENTATION**

#### **BEAT WISE** - **Myocarditis**

In [5]:
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

Z = MC

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/HODMD Paper/Features_Beat_NoArg_Myocarditis.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

Patient: patient269 Audio ID: s0508_re
n: 0, Stable: 1, Unstable: 0
n: 1, Stable: 1, Unstable: 0
n: 2, Stable: 1, Unstable: 0
n: 3, Stable: 1, Unstable: 0
n: 4, Stable: 1, Unstable: 0
n: 5, Stable: 1, Unstable: 0
n: 6, Stable: 1, Unstable: 0
n: 7, Stable: 1, Unstable: 0
n: 8, Stable: 1, Unstable: 0
n: 9, Stable: 1, Unstable: 0
n: 10, Stable: 1, Unstable: 0
n: 11, Stable: 1, Unstable: 0
n: 12, Stable: 1, Unstable: 0
n: 13, Stable: 2, Unstable: 0
n: 14, Stable: 1, Unstable: 0
n: 15, Stable: 1, Unstable: 0
n: 16, Stable: 1, Unstable: 0
n: 17, Stable: 1, Unstable: 0
n: 18, Stable: 1, Unstable: 0
n: 19, Stable: 1, Unstable: 0
n: 20, Stable: 1, Unstable: 0
n: 21, Stable: 1, Unstable: 0
n: 22, Stable: 1, Unstable: 0
n: 23, Stable: 1, Unstable: 0
n: 24, Stable: 1, Unstable: 0
n: 25, Stable: 1, Unstable: 0
n: 26, Stable: 1, Unstable: 0
n: 27, Stable: 1, Unstable: 0
n: 28, Stable: 1, Unstable: 0
n: 29, Stable: 1, Unstable: 0
n: 30, Stable: 2, Unstable: 0
n: 31, Stable: 2, Unstable: 0
n: 32, Stab

ZeroDivisionError: division by zero

#### **BEAT WISE** - **Valvular heart disease**

In [ ]:
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

Z = VH

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/HODMD Paper/Features_Beat_NoArg_Valvular heart disease.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

Patient: patient106 Audio ID: s0030_re
Patient: patient110 Audio ID: s0003_re
n: 0, Stable: 44, Unstable: 42
Patient: patient107 Audio ID: s0199_re
n: 1, Stable: 54, Unstable: 30
n: 0, Stable: 64, Unstable: 24
n: 0, Stable: 40, Unstable: 54
n: 2, Stable: 57, Unstable: 30
Patient: patient114 Audio ID: s0012_re
n: 1, Stable: 58, Unstable: 42
n: 1, Stable: 52, Unstable: 38
n: 2, Stable: 49, Unstable: 51
n: 3, Stable: 59, Unstable: 28
n: 2, Stable: 58, Unstable: 33
n: 0, Stable: 56, Unstable: 36
n: 3, Stable: 43, Unstable: 48
n: 4, Stable: 50, Unstable: 34
n: 3, Stable: 46, Unstable: 54
n: 1, Stable: 60, Unstable: 31
n: 4, Stable: 54, Unstable: 34
n: 4, Stable: 47, Unstable: 48
n: 5, Stable: 42, Unstable: 40
n: 2, Stable: 52, Unstable: 35
n: 5, Stable: 48, Unstable: 42
n: 5, Stable: 44, Unstable: 54
n: 6, Stable: 53, Unstable: 30
n: 3, Stable: 49, Unstable: 41
n: 6, Stable: 67, Unstable: 24
n: 6, Stable: 40, Unstable: 60
n: 7, Stable: 40, Unstable: 42
n: 4, Stable: 56, Unstable: 37
n: 7, S

#### **BEAT WISE** - **Myocardial hypertrophy**

In [ ]:
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

Z = MH

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/HODMD Paper/Features_Beat_NoArg_Myocardial hypertrophy.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

#### **BEAT WISE** - **Dysrhythmia**

In [ ]:
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

Z = DA

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/HODMD Paper/Features_Beat_NoArg_Dysrhythmia.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

Patient: patient112 Audio ID: s0169_re
Patient: patient147 Audio ID: s0211_re
Patient: patient168 Audio ID: s0032_re
Patient: patient151 Audio ID: s0206_re
n: 0, Stable: 89, Unstable: 6
n: 0, Stable: 73, Unstable: 20
n: 0, Stable: 51, Unstable: 40
Patient: patient187 Audio ID: s0207_re
n: 0, Stable: 75, Unstable: 18
n: 1, Stable: 74, Unstable: 20
n: 1, Stable: 66, Unstable: 28
n: 0, Stable: 58, Unstable: 36
n: 1, Stable: 57, Unstable: 33
n: 2, Stable: 68, Unstable: 26
n: 2, Stable: 58, Unstable: 40
n: 1, Stable: 76, Unstable: 12
n: 1, Stable: 46, Unstable: 50
n: 2, Stable: 56, Unstable: 40
n: 3, Stable: 56, Unstable: 38
n: 3, Stable: 78, Unstable: 12
n: 2, Stable: 56, Unstable: 43
n: 2, Stable: 72, Unstable: 14
n: 4, Stable: 78, Unstable: 14
n: 4, Stable: 85, Unstable: 6
n: 3, Stable: 60, Unstable: 39
n: 3, Stable: 72, Unstable: 26
n: 3, Stable: 55, Unstable: 46
n: 5, Stable: 82, Unstable: 16
Patient: patient113 Audio ID: s0018lre
n: 5, Stable: 84, Unstable: 12
n: 4, Stable: 84, Unstab

#### **BEAT WISE** - **Bundle branch block**

In [ ]:
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

Z = BBB

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/HODMD Paper/Features_Beat_NoArg_Bundle branch block.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

Patient: patient175 Audio ID: s0009_re
n: 0, Stable: 39, Unstable: 52
n: 1, Stable: 51, Unstable: 44
Patient: patient204 Audio ID: s0425_re
Patient: patient199 Audio ID: s0404lre
n: 2, Stable: 60, Unstable: 38
Patient: patient203 Audio ID: s0424_re
Patient: patient206 Audio ID: s0427_re
n: 0, Stable: 64, Unstable: 30
Patient: patient171 Audio ID: s0364lre
n: 0, Stable: 98, Unstable: 0
n: 0, Stable: 82, Unstable: 12
n: 3, Stable: 50, Unstable: 52
n: 0, Stable: 33, Unstable: 52
n: 0, Stable: 65, Unstable: 32
n: 1, Stable: 81, Unstable: 12
n: 1, Stable: 71, Unstable: 22
n: 1, Stable: 36, Unstable: 48
n: 4, Stable: 64, Unstable: 42
Patient: patient209 Audio ID: s0431_re
n: 1, Stable: 74, Unstable: 22
n: 1, Stable: 62, Unstable: 35
n: 2, Stable: 50, Unstable: 45
n: 2, Stable: 35, Unstable: 48
n: 5, Stable: 57, Unstable: 44
n: 2, Stable: 88, Unstable: 6
n: 2, Stable: 78, Unstable: 18
n: 0, Stable: 66, Unstable: 24
n: 2, Stable: 58, Unstable: 40
n: 3, Stable: 83, Unstable: 14
n: 3, Stable: 45

#### **BEAT WISE** - **Cardiomyopathy/Heart failure**

In [ ]:
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

Z = HF

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/HODMD Paper/Features_Beat_NoArg_Cardiomyopathy_HeartFailure.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

Patient: patient167 Audio ID: s0200_re
Patient: patient118 Audio ID: s0183_re
Patient: patient115 Audio ID: s0023_re
Patient: patient129 Audio ID: s0189_re
Patient: patient123 Audio ID: s0224_re
n: 0, Stable: 62, Unstable: 29
n: 0, Stable: 70, Unstable: 18
n: 0, Stable: 39, Unstable: 54
n: 0, Stable: 52, Unstable: 40
n: 0, Stable: 58, Unstable: 36
n: 1, Stable: 79, Unstable: 19
n: 1, Stable: 56, Unstable: 26
n: 1, Stable: 32, Unstable: 58
n: 1, Stable: 57, Unstable: 40
n: 1, Stable: 69, Unstable: 22
n: 2, Stable: 67, Unstable: 8
n: 2, Stable: 79, Unstable: 14
n: 2, Stable: 44, Unstable: 46
n: 2, Stable: 48, Unstable: 42
n: 2, Stable: 68, Unstable: 22
n: 3, Stable: 71, Unstable: 14
n: 3, Stable: 55, Unstable: 36
n: 3, Stable: 81, Unstable: 20
n: 3, Stable: 39, Unstable: 52
n: 3, Stable: 58, Unstable: 33
n: 4, Stable: 45, Unstable: 40
n: 4, Stable: 41, Unstable: 48
n: 4, Stable: 44, Unstable: 46
n: 4, Stable: 67, Unstable: 27
n: 5, Stable: 70, Unstable: 10
n: 4, Stable: 67, Unstable: 22


#### **BEAT WISE** - **Myocardial infarction**

In [ ]:
warnings.simplefilter("ignore", RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

Z = MI[:60]
print(len(Z))

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/HODMD Paper/Features_Beat_NoArg_Myocardial infarction.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

60
Patient: patient111 Audio ID: s0203_re
n: 0, Stable: 55, Unstable: 46
Patient: patient022 Audio ID: s0074lre
n: 1, Stable: 59, Unstable: 37
n: 2, Stable: 45, Unstable: 50
n: 3, Stable: 40, Unstable: 56
Patient: patient020 Audio ID: s0069lre
n: 0, Stable: 56, Unstable: 43
n: 4, Stable: 48, Unstable: 42
n: 0, Stable: 70, Unstable: 22
n: 1, Stable: 41, Unstable: 59
Patient: patient003 Audio ID: s0017lre
n: 2, Stable: 67, Unstable: 33
n: 5, Stable: 52, Unstable: 40
n: 0, Stable: 76, Unstable: 18
Patient: patient090 Audio ID: s0356lre
Patient: patient071 Audio ID: s0236lre
n: 1, Stable: 82, Unstable: 14
n: 6, Stable: 51, Unstable: 40
Patient: patient013 Audio ID: s0051lre
n: 3, Stable: 72, Unstable: 28
n: 1, Stable: 51, Unstable: 34
n: 0, Stable: 65, Unstable: 26
n: 0, Stable: 79, Unstable: 16n: 7, Stable: 37, Unstable: 50

n: 2, Stable: 78, Unstable: 14
n: 0, Stable: 61, Unstable: 38
n: 1, Stable: 76, Unstable: 16
n: 8, Stable: 33, Unstable: 56
n: 2, Stable: 59, Unstable: 34
n: 4, Stabl

#### **BEAT WISE** - **Health Control**

In [ ]:
Z = HC
print(len(Z))

with Pool() as pool:
    # prepare arguments
    items = [(Z[i][0],Z[i][1]) for i in range(len(Z))]
    # issues tasks to process pool
    result = pool.starmap_async(Patient_FeatureExtraction_Beat, items)
    # iterate results
    for result in result.get():
        f = open(dir + "/FEATURE_EXTRACTION/HODMD Paper/Features_Beat_NoArg_Health Control.csv", "a")
        for line in result:
            f.write(line + '\n')
        f.close()

pool.close()
pool.join()

52
Patient: patient267 Audio ID: s0504_re
Patient: patient234 Audio ID: s0460_re
Patient: patient244 Audio ID: s0473_re
Patient: patient266 Audio ID: s0502_re
n: 0, Stable: 83, Unstable: 6
n: 0, Stable: 91, Unstable: 8
n: 0, Stable: 54, Unstable: 34
n: 1, Stable: 83, Unstable: 10
Patient: patient241 Audio ID: s0470_re
n: 0, Stable: 73, Unstable: 26
n: 1, Stable: 70, Unstable: 28
Patient: patient229 Audio ID: s0452_re
n: 1, Stable: 94, Unstable: 3
n: 2, Stable: 80, Unstable: 10
Patient: patient174 Audio ID: s0325lre
Patient: patient182 Audio ID: s0308lre
n: 1, Stable: 54, Unstable: 42
n: 0, Stable: 97, Unstable: 8
Patient: patient263 Audio ID: s0499_re
n: 0, Stable: 67, Unstable: 22
n: 2, Stable: 61, Unstable: 28
n: 3, Stable: 66, Unstable: 20
n: 0, Stable: 79, Unstable: 22
n: 2, Stable: 80, Unstable: 14
Patient: patient240 Audio ID: s0468_re
n: 0, Stable: 42, Unstable: 52
Patient: patient248 Audio ID: s0481_re
n: 1, Stable: 72, Unstable: 16
n: 2, Stable: 70, Unstable: 25
n: 0, Stable: 